In [6]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string

# --------------------------------------
# Load the JSON data
# --------------------------------------
with open("gallery_data_augmented.json", "r") as f:
    data = json.load(f)

df = pd.json_normalize(data)

# --------------------------------------
# Create results folder if it doesn't exist
# --------------------------------------
results_dir = "./results"
os.makedirs(results_dir, exist_ok=True)

# --------------------------------------
# Clean and Extract Sentiment Labels
# --------------------------------------
sentiment_raw = df["sentiment"].dropna()

sentiment_cleaned = (
    sentiment_raw
    .str.extract(r"\*\*(.*?)\*\*")[0]
    .str.strip()
    .str.strip(string.punctuation)
    .str.capitalize()
)

# Custom sentiment order: most negative to positive
sentiment_order = ["Tragic", "Distressing", "Concerned", "Hopeful"]

# --------------------------------------
# Load and Clean Loss and Resilience Levels
# --------------------------------------
loss_levels = df["lossLevel"].dropna().astype(int)
resilience_levels = df["resilienceLevel"].dropna().astype(int)

# Set Seaborn style
sns.set(style="whitegrid")

# --------------------------------------
# Plot 1: Sentiment Histogram (save to PDF)
# --------------------------------------
plt.figure(figsize=(8, 5))
sns.countplot(
    x=sentiment_cleaned,
    hue=sentiment_cleaned,
    order=sentiment_order,
    palette="Blues",
    legend=False
)
plt.title("Histogram of Sentiment Levels")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(results_dir, "sentiment_histogram.pdf"))
plt.close()

# --------------------------------------
# Plot 2: Loss Level Histogram (save to PDF)
# --------------------------------------
plt.figure(figsize=(6, 4))
sns.countplot(x=loss_levels, hue=loss_levels, palette="Reds", legend=False)
plt.title("Histogram of Loss Levels")
plt.xlabel("Loss Level (1=None, 2=Moderate, 3=High)")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig(os.path.join(results_dir, "loss_histogram.pdf"))
plt.close()

# --------------------------------------
# Plot 3: Resilience Level Histogram (save to PDF)
# --------------------------------------
plt.figure(figsize=(6, 4))
sns.countplot(x=resilience_levels, hue=resilience_levels, palette="Greens", legend=False)
plt.title("Histogram of Resilience Levels")
plt.xlabel("Resilience Level (1=None, 2=Moderate, 3=High)")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig(os.path.join(results_dir, "resilience_histogram.pdf"))
plt.close()

# --------------------------------------
# Create and Save Joint Matrix
# --------------------------------------
matrix = pd.crosstab(loss_levels, resilience_levels)
print("\nJoint Matrix: Loss Level vs Resilience Level")
print(matrix)

matrix_csv_path = os.path.join(results_dir, "loss_vs_resilience_matrix.csv")
matrix.to_csv(matrix_csv_path)



Joint Matrix: Loss Level vs Resilience Level
resilienceLevel  1   2    3
lossLevel                  
1                1   0    1
2                2   5    6
3                8  50  240
